In [12]:
import pandas as pd
import numpy as np

import requests
import re
from bs4 import BeautifulSoup
import difflib
from tqdm import tqdm
import os
import time

Preparation: Get paths to the pdfs

In [13]:
path_constant_part = "C:/Users/Jonas Hummel/Documents/Studium - PhD Informatik/Survey Paper Interaction/Study PDFs/"

pdf_path_dict = {
    1: '33_weisenberger.pdf',
    2: '714_brewster.pdf',
    3: '32_metzger.pdf',
    4: '40_buil.pdf', 
    5: '84_buil.pdf', 
    6: '61_manabe.pdf',
    7: '623_simpson.pdf',
    8: '184_tamaki.pdf',
    9: '624_simpson.pdf',
    10: '253_gamper.pdf',
    11: '23_manabe.pdf',
    12: '85_matsumura.pdf',
    13: '62_sano.pdf',
    14: '94_tessendorf.pdf',
    15: '25_akiyama.pdf',
    16: '20_manabe.pdf',
    17: '1_lissermann.pdf',
    18: '93_sahni.pdf',
    19: '7_bedri.pdf',
    20: '190_bleichner.pdf',
    21: '92_norton.pdf',
    22: '188_wang.pdf',
    23: '19_weigel.pdf',
    24: '63_ashbrook.pdf',
    25: '34_laput.pdf',
    26: '69_merrill.pdf',
    27: '43_nguyen.pdf',
    28: '17_ando.pdf',
    29: '206_dim.pdf',
    30: '22_kikuchi.pdf',
    31: '64_maag.pdf',
    32: '12_matthies.pdf',
    33: '111_wang.pdf',
    34: '117_ahn.pdf',
    35: '177_carioli.pdf',
    36: '91_favre_felix.pdf',
    37: '31_huang.pdf',
    38: '18_lee.pdf',
    39: '610_min.pdf',
    40: '65_nguyen.pdf',
    41: '60_taniguchi.pdf',
    42: '8_amesaka.pdf',
    43: '80_ferlini.pdf',
    44: '77_hoelzemann.pdf',
    45: '106_lee.pdf',
    46: '14_lee.pdf',
    47: '2_nasser.pdf',
    48: '78_odoemelem.pdf',
    49: '15_shirota.pdf',
    50: '620_vega_galvez.pdf',
    51: '27_yan.pdf',
    52: '223_cao.pdf',
    53: '234_chen.pdf',
    54: '212_chen.pdf',
    55: '149_kaveh.pdf',
    56: '16_prakash.pdf',
    57: '26_xu.pdf',
    58: '611_yang.pdf',
    59: '229_fan.pdf',
    60: '247_ferlini.pdf',
    61: '258_gashi.pdf',
    62: '217_hashem.pdf',
    63: '622_islam.pdf',
    64: '226_jin.pdf',
    65: '618_kakaraparthi.pdf',
    66: '213_khanna.pdf',
    67: '224_laporte.pdf',
    68: '626_ma.pdf',
    69: '259_nasser.pdf',
    70: '628_peng.pdf',
    71: '227_röddiger.pdf',
    72: '221_sun.pdf',
    73: '231_verma.pdf',
    74: '616_wu.pdf',
    75: '608_alkiek.pdf',
    76: '614_bi.pdf',
    77: '452_bi.pdf',
    78: '345_choi.pdf',
    79: '627_futami.pdf',
    80: '315_jin.pdf',
    81: '278_li.pdf',
    82: '512_song.pdf',
    83: '296_rateau.pdf',
    84: '307_srivastava.pdf',
    85: '309_wang.pdf',
    86: '306_wang.pdf',
    87: '388_yang.pdf',
    88: '284_alkiek.pdf',
    89: '344_chugh.pdf',
    90: '342_jin.pdf',
    91: '376_li.pdf',
    92: '308_panda.pdf',
    93: '443_paul.pdf',
    94: '277_stanke.pdf',
    95: '461_yi.pdf',
    96: '619_zhang.pdf',
    97: '292_zhang.pdf',
    98: '713_zhang.pdf',
    99: '516_zhu.pdf',
    100: '346_dong.pdf',
    101: '498_ge.pdf',
    102: '482_hu.pdf',
    103: '509_hu.pdf',
    104: '625_lepold.pdf',
    105: '449_ronco.pdf',
    106: '635_sato.pdf',
    107: '291_shojaeifard.pdf',
    108: '640_srivastava.pdf',
    109: '275_shimon.pdf',
    110: '654_srivastava.pdf',
    111: '506_sun.pdf',
    112: '639_suzuki.pdf',
    113: '286_wang.pdf',
    114: '532_wang.pdf',
    115: '676_wang.pdf',
    116: '675_xie.pdf',
    117: '354_yang.pdf',
    118: '527_yi.pdf',
}

pdf_path_dict_keys = list(pdf_path_dict.keys())

In [3]:
# Dictionary to track valid and invalid paths
path_status = {
    'valid': [],
    'invalid': []
}

# Loop through all entries in pdf_path_dict to check if files exist
print("Checking PDF file paths...")
for id_num, pdf_name in tqdm(pdf_path_dict.items()):
    full_path = os.path.join(path_constant_part, pdf_name)
    
    if os.path.isfile(full_path):
        path_status['valid'].append((id_num, pdf_name))
    else:
        path_status['invalid'].append((id_num, pdf_name))

# Print results
print(f"\nResults:")
print(f"- Valid paths: {len(path_status['valid'])}/{len(pdf_path_dict)}")
print(f"- Invalid paths: {len(path_status['invalid'])}/{len(pdf_path_dict)}")

# Print details of invalid paths if any exist
if path_status['invalid']:
    print("\nInvalid paths:")
    for id_num, pdf_name in path_status['invalid']:
        print(f"ID {id_num}: {path_constant_part}{pdf_name}")

Checking PDF file paths...


100%|██████████| 118/118 [00:00<00:00, 1790.62it/s]


Results:
- Valid paths: 118/118
- Invalid paths: 0/118


Extract the Authors from the PDFs and find matching authors

In [14]:
# Set display options to show all content
pd.set_option('display.max_colwidth', None)  # Show full column content
pd.set_option('display.width', 1000)  # Wide display

df_id_authors = pd.DataFrame(columns=['ID', 'Authors'])

for i in tqdm(range(119)): 
    if i in pdf_path_dict_keys:
        
        pdf_path = f'{path_constant_part}{pdf_path_dict[i]}'
        
        # Read the file content within the with block
        with open(pdf_path, "rb") as pdf_file:
            pdf_content = pdf_file.read()
        
        # Use the content in the request
        files = {"input": ("filename.pdf", pdf_content, "application/pdf")}
            
        response_header = requests.post("http://localhost:8070/api/processHeaderDocument", files=files)
        response_string = response_header.text  # TEI XML output
        
        author_extract = re.search(r'author\s*=\s*\{(.*?)\}', response_string)
        if author_extract:
            author_string = author_extract.group(1)
            # Split by 'and' to get individual authors
            authors_raw = [author.strip() for author in author_string.split(' and ')]
            
            # Reverse "Last, First" to "First Last" format
            authors = []
            for author in authors_raw:
                if ',' in author:
                    last_name, first_name = author.split(',', 1)
                    full_name = f"{first_name.strip()} {last_name.strip()}"
                    authors.append(full_name)
                else:
                    # If there's no comma, keep the name as is
                    authors.append(author)
                    
        df_id_authors.loc[len(df_id_authors)] = [i, authors]
            

100%|██████████| 119/119 [02:49<00:00,  1.42s/it]


In [15]:
df_id_authors


,ID,Authors
0,1,"[Janet Weisenbercer, Arnold Heidbreder, James Miller]"
1,2,"[Stephen Brewster, Joanna Lumsden, Marek Bell, Malcolm Hall, Stuart Tasker]"
2,3,"[Christian Metzger, Matt Anderson, Thad Starner]"
3,4,"[Vincent Buil, Gerard Hollemans]"
4,5,"[Vincent Buil, Gerard Hollemans, Sander Van De Wijdeven]"
...,...,...
113,114,"[Yong Wang, Tianyu Yang, Chunxiao Wang, Pengfei Hu, Yiran Shen, Feng Li]"
114,115,"[Yilin Wang, Zi Wang, Jie Yang]"
115,116,"[Yanze Xie, Mengzhen Gao, Xiaoning Liu, Shuo Huang, Helei Cui, Zhiwen Yu, Bin Guo]"
116,117,"[Yongjie Yang, Tao Chen, Yujing Huang, Xiuzhen Guo, Longfei Shangguan, Longfei Shang]"


In [16]:
np.array((df_id_authors['ID'].to_numpy(), df_id_authors['Authors'].to_numpy())).T

array([[1,
        list(['Janet Weisenbercer', 'Arnold Heidbreder', 'James Miller'])],
       [2,
        list(['Stephen Brewster', 'Joanna Lumsden', 'Marek Bell', 'Malcolm Hall', 'Stuart Tasker'])],
       [3, list(['Christian Metzger', 'Matt Anderson', 'Thad Starner'])],
       [4, list(['Vincent Buil', 'Gerard Hollemans'])],
       [5,
        list(['Vincent Buil', 'Gerard Hollemans', 'Sander Van De Wijdeven'])],
       [6, list(['Hiroyuki Manabe', 'Masaaki Fukumoto'])],
       [7, list(['Tyler Simpson', 'Colin Broughton', 'Michel Gauthier'])],
       [8, list(['Emi Tamaki', 'Takashi Miyaki', 'Jun Rekimoto'])],
       [9,
        list(['Tyler Simpson', 'Michel Gauthier', 'Arthur Prochazka'])],
       [10, list(['Hannes Gamper', 'Sakari Tervo', 'Tapio Lokki'])],
       [11, list(['Hiroyuki Manabe', 'Masaaki Fukumoto'])],
       [12,
        list(['Kohei Matsumura', 'Daisuke Sakamoto', 'Takeo Igarashi'])],
       [13, list(['Akane Sano', 'Takashi Tomita', 'Haruo Oba'])],
       [14,
 

In [17]:
# Manual replacements
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 1].index[0], 'Authors'] = ['Janet Weisenberger', 'Arnold Heidbreder', 'James Miller']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 20].index[0], 'Authors'] = ['Martin Bleichner', 'Micha Lundbeck', 'Matthias Selisky', 'Falk Minow', 'Manuela Jaeger', 'Reiner Emkes', 'Stefan Debener', 'Maarten De Vos', 'Neuropsychology Lab']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 23].index[0], 'Authors'] = ['Martin Weigel', 'Tong Lu', 'Gilles Bailly', 'Antti Oulasvirta', 'Carmel Majidi', 'Juergen Steimle']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 25].index[0], 'Authors'] = ['Gierad Xiang', 'Anthony Chen', 'Chris Harrison']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 27].index[0], 'Authors'] = ['Anh Nguyen', 'Raghda Alqurashi', 'Zohreh Raghebi', 'Farnoush Banaei-Kashani', 'Ann Halbower', 'Tam Vu']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 36].index[0], 'Authors'] = ['Antoine Favre-Felix', 'Carina Graversen', 'Renskje Hietkamp', 'Dau Torsten', 'Thomas Lunner']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 96].index[0], 'Authors'] = ['Yuke Zhang', 'Ken Takaki', 'Hiroaki Murakami', 'Takuya Sasatani', 'Yoshihiro Kawahara']
df_id_authors.at[df_id_authors[df_id_authors['ID'] == 113].index[0], 'Authors'] = ['Zi Wang', 'Yilin Wang', 'Jie Yang']


In [18]:
# Extract all authors into a flat list
all_authors = []
for authors_list in df_id_authors['Authors']:
    all_authors.extend(authors_list)

# Count occurrences of each author
author_counts = pd.Series(all_authors).value_counts().reset_index()
author_counts.columns = ['Author', 'Count']

# Sort first by count (descending) and then by author name (ascending)
author_counts = author_counts.sort_values(['Count', 'Author'], ascending=[False, True])

# Display the results
author_counts

,Author,Count
0,Phuc Nguyen,6
2,Cheng Zhang,4
1,Tanmay Srivastava,4
9,Alessandro Montanari,3
10,Andrea Ferlini,3
...,...,...
409,Zihang Huang,1
407,Zimu Zhou,1
212,Zisu Li,1
278,Zohreh Raghebi,1


In [20]:
coauthor_matrix = pd.DataFrame(0, index=np.arange(1, 119), columns=np.arange(1, 119))

ids = np.arange(1, 119)

# Create a dictionary mapping paper IDs to their authors for faster lookup
id_to_authors = {row['ID']: set(row['Authors']) for _, row in df_id_authors.iterrows()}

# Function to check if two author names are similar (allowing for small differences)
def are_similar_names(name1, name2, max_distance=1):
    # Calculate Levenshtein distance between names
    distance = sum(1 for a, b in zip(name1, name2) if a != b) + abs(len(name1) - len(name2))
    return distance <= max_distance

# Iterate through each pair of papers (only checking each pair once)
for id_i in ids:
    authors_i = id_to_authors[id_i]
        
    for id_j in range(id_i+1, len(ids)+1):  # Start from i+1 to avoid duplicate checks
        authors_j = id_to_authors[id_j]
        
        # Check for exact author matches first
        if len(authors_i.intersection(authors_j)) > 0:
            coauthor_matrix.loc[id_i, id_j] = 1
            coauthor_matrix.loc[id_j, id_i] = 1  # Matrix is symmetric
            continue
        
        # If no exact match, check for similar author names
        for author_i in authors_i:
            for author_j in authors_j:
                if are_similar_names(author_i, author_j, max_distance=2):
                    # Ask user to confirm if these are the same author
                    print(f"\nPossible author match between papers {id_i} and {id_j}:")
                    print(f"  Author in paper {id_i}: '{author_i}'")
                    print(f"  Author in paper {id_j}: '{author_j}'")
                    confirm = input("Are these the same author? (y/n): ").strip().lower() == 'y'
                    
                    if confirm:
                        coauthor_matrix.loc[id_i, id_j] = 1
                        coauthor_matrix.loc[id_j, id_i] = 1  # Matrix is symmetric
                        print(f"Added co-authorship between papers {id_i} and {id_j}")
                        # No need to check other authors once we've established a connection
                        break
            else:
                continue  # Continue if the inner loop wasn't broken
            break  # Break the outer loop if the inner loop was broken

coauthor_matrix.to_csv('interconnections_datasets/coauthor_matrix.csv')

# Display first few rows and columns
coauthor_matrix.head()


Possible author match between papers 21 and 45:
  Author in paper 21: 'Woosik Lee'
  Author in paper 45: 'Woojin Lee'

Possible author match between papers 72 and 95:
  Author in paper 72: 'Feng Tian'
  Author in paper 95: 'Feng Jiang'

Possible author match between papers 72 and 118:
  Author in paper 72: 'Feng Tian'
  Author in paper 118: 'Feng Jiang'

Possible author match between papers 98 and 117:
  Author in paper 98: 'Hao Chen'
  Author in paper 117: 'Tao Chen'

Possible author match between papers 101 and 102:
  Author in paper 101: 'Qian Zhang'
  Author in paper 102: 'Qibo Zhang'


,1,2,3,4,5,6,7,8,9,10,...,109,110,111,112,113,114,115,116,117,118
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Get the Citation Matrix of out of the PDFs

In [4]:
# DF with ID and bibtex
df_id_bibtex = pd.read_excel('interconnections_datasets/bibtex_mapping_of_ids.xlsx')

In [14]:
# Function to extract references from a paper using GROBID and convert to BibTeX
def extract_references_to_bibtex(pdf_path):
    """Extract references from PDF using GROBID and convert to BibTeX format"""
    references = []
    
    try:
        # Read PDF content
        with open(pdf_path, "rb") as pdf_file:
            pdf_content = pdf_file.read()
        
        # Request GROBID to process references
        files = {"input": ("document.pdf", pdf_content, "application/pdf")}
        response = requests.post(
            "http://localhost:8070/api/processReferences", 
            files=files, 
            timeout=300
        )
        
        if response.status_code != 200:
            print(f"Error from GROBID API: {response.status_code}")
            return []
            
        # Parse XML response
        soup = BeautifulSoup(response.text, 'xml')
        
        for i, bibl in enumerate(soup.find_all('biblStruct')):
            # Extract key citation components
            ref_id = bibl.get('xml:id', f'ref_{i}')
            
            # Authors
            authors = []
            for author_tag in bibl.find_all('author'):
                person = author_tag.find('persName')
                if person:
                    surname = person.find('surname')
                    forename = person.find('forename')
                    
                    if surname:
                        author_name = surname.text
                        if forename:
                            author_name = f"{author_name}, {forename.text}"
                        authors.append(author_name)
            
            # Title
            title = ""
            title_tag = bibl.find('title', {'level': 'a'})
            if title_tag:
                title = title_tag.text.strip()
            
            # Year
            year = ""
            date_tag = bibl.find('date', {'type': 'published'})
            if date_tag and date_tag.get('when'):
                year = date_tag.get('when').split('-')[0]  # Extract year from date
            
            # Journal/Conference
            journal = ""
            journal_tag = bibl.find('title', {'level': 'j'})
            if journal_tag:
                journal = journal_tag.text.strip()
            else:
                book_tag = bibl.find('title', {'level': 'm'})
                if book_tag:
                    journal = book_tag.text.strip()
            
            # Volume, Issue, Pages
            volume = ""
            vol_tag = bibl.find('biblScope', {'unit': 'volume'})
            if vol_tag:
                volume = vol_tag.text.strip()
            
            issue = ""
            issue_tag = bibl.find('biblScope', {'unit': 'issue'})
            if issue_tag:
                issue = issue_tag.text.strip()
            
            pages = ""
            pages_from_tag = bibl.find('biblScope', {'unit': 'page', 'from': True})
            pages_to_tag = bibl.find('biblScope', {'unit': 'page', 'to': True})
            if pages_from_tag and pages_to_tag:
                pages = f"{pages_from_tag.get('from')}--{pages_to_tag.get('to')}"
            elif pages_from_tag:
                pages = pages_from_tag.get('from')
            
            # DOI
            doi = ""
            doi_tag = bibl.find('idno', {'type': 'DOI'})
            if doi_tag:
                doi = doi_tag.text.strip()
            
            # Create BibTeX entry
            bibtex_id = f"grobid_{ref_id.replace('b', '')}"
            bibtex_str = f"@article{{{bibtex_id},\n"
            
            if authors:
                bibtex_str += f"  author = {{{' and '.join(authors)}}},\n"
            if title:
                bibtex_str += f"  title = {{{title}}},\n"
            if journal:
                bibtex_str += f"  journal = {{{journal}}},\n"
            if year:
                bibtex_str += f"  year = {{{year}}},\n"
            if volume:
                bibtex_str += f"  volume = {{{volume}}},\n"
            if issue:
                bibtex_str += f"  number = {{{issue}}},\n"
            if pages:
                bibtex_str += f"  pages = {{{pages}}},\n"
            if doi:
                bibtex_str += f"  doi = {{{doi}}},\n"
                
            bibtex_str += "}"
            
            # Create structured citation data
            citation = {
                'bibtex': bibtex_str,
                'authors': authors,
                'title': title,
                'year': year,
                'journal': journal,
                'doi': doi,
                'raw_xml': str(bibl),
                'author_last_names': [a.split(',')[0].lower().strip() if ',' in a else a.split()[-1].lower().strip() 
                                      for a in authors]
            }
            
            references.append(citation)
            
    except Exception as e:
        print(f"Error extracting references: {str(e)}")
    
    return references

In [15]:
# Function to normalize BibTeX entries for comparison
def normalize_bibtex_for_comparison(bibtex_str):
    """Extract key information from BibTeX for comparison"""
    result = {
        'authors': [],
        'title': '',
        'year': '',
        'journal': '',
        'doi': '',
        'author_last_names': []
    }
    
    # Extract authors
    author_match = re.search(r'author\s*=\s*\{(.*?)\}', bibtex_str, re.DOTALL)
    if author_match:
        authors_str = author_match.group(1)
        authors = [a.strip() for a in authors_str.split(' and ')]
        result['authors'] = authors
        # Extract last names
        result['author_last_names'] = [a.split(',')[0].lower().strip() if ',' in a 
                                      else a.split()[-1].lower().strip() 
                                      for a in authors]
    
    # Extract title
    title_match = re.search(r'title\s*=\s*\{(.*?)\}', bibtex_str, re.DOTALL)
    if title_match:
        result['title'] = re.sub(r'[\{\}]', '', title_match.group(1).lower())
    
    # Extract year
    year_match = re.search(r'year\s*=\s*\{?(\d{4})\}?', bibtex_str)
    if year_match:
        result['year'] = year_match.group(1)
    
    # Extract journal or booktitle
    journal_match = re.search(r'journal\s*=\s*\{(.*?)\}', bibtex_str, re.DOTALL)
    if not journal_match:
        journal_match = re.search(r'booktitle\s*=\s*\{(.*?)\}', bibtex_str, re.DOTALL)
    if journal_match:
        result['journal'] = journal_match.group(1).lower()
    
    # Extract DOI
    doi_match = re.search(r'doi\s*=\s*\{(.*?)\}', bibtex_str, re.DOTALL)
    if doi_match:
        result['doi'] = doi_match.group(1).lower()
    
    return result

In [16]:
# Function to match citation with corpus papers
def match_citation_to_corpus(citation_data, corpus_entries, citing_paper_id, threshold=0.5):
    """Match a citation to papers in the corpus"""
    matches = []
    
    # Extract normalized data from citation
    citation_title = citation_data.get('title', '').lower()
    citation_title = re.sub(r'[^\w\s]', '', citation_title)
    citation_year = citation_data.get('year', '')
    citation_authors = citation_data.get('author_last_names', [])
    citation_doi = citation_data.get('doi', '').lower()
    
    for paper_id, paper_data in corpus_entries.items():
        # Skip self-citations (ADDED)
        if paper_id == citing_paper_id:
            continue
            
        score = 0
        max_score = 0
        match_details = {}
        
        # Extract normalized data from corpus entry
        paper_title = paper_data.get('title', '').lower()
        paper_title = re.sub(r'[^\w\s]', '', paper_title)
        paper_year = paper_data.get('year', '')
        paper_authors = paper_data.get('author_last_names', [])
        paper_doi = paper_data.get('doi', '').lower()
        
        # Match by DOI (highest confidence)
        if citation_doi and paper_doi and citation_doi == paper_doi:
            return [{'paper_id': paper_id, 'score': 1.0, 'match_type': 'doi'}]
        
        # Match by authors (up to 2 points)
        if citation_authors and paper_authors:
            max_score += 2
            matching_authors = set(citation_authors) & set(paper_authors)
            if matching_authors:
                author_score = min(2, len(matching_authors))
                score += author_score
                match_details['matching_authors'] = list(matching_authors)
        
        # Match by year (1 point)
        if citation_year and paper_year:
            max_score += 1
            if citation_year == paper_year:
                score += 1
                match_details['year_match'] = True
        
        # Match by title (5 points max)
        if citation_title and paper_title:
            max_score += 5
            
            # Calculate title similarity
            title_similarity = difflib.SequenceMatcher(None, citation_title, paper_title).ratio()
            
            # Check for title containment (special case)
            if (len(citation_title) > 10 and len(paper_title) > 10):
                if citation_title in paper_title or paper_title in citation_title:
                    title_similarity = max(title_similarity, 0.8)
            
            title_score = title_similarity * 5
            score += title_score
            match_details['title_similarity'] = title_similarity
        
        # Calculate final score
        if max_score > 0:
            final_score = score / max_score
            if final_score >= threshold:
                matches.append({
                    'paper_id': paper_id,
                    'score': final_score,
                    'details': match_details
                })
    
    # Sort matches by score
    matches.sort(key=lambda x: x['score'], reverse=True)
    return matches

In [17]:
# Function to build citation network with Excel-based confirmation
def build_citation_network_with_excel(df_id_bibtex, pdf_path_dict, path_constant_part):
    """Build a citation network from papers and their references with Excel-based confirmation"""
    print("Preparing citation network analysis...")
    
    # Step 1: Create normalized corpus entries from BibTeX data
    print("Normalizing corpus BibTeX entries...")
    corpus_entries = {}
    
    for _, row in df_id_bibtex.iterrows():
        paper_id = row['ID']
        bibtex_str = row['Bibtex'] if isinstance(row['Bibtex'], str) else ""
        
        if bibtex_str:
            normalized_data = normalize_bibtex_for_comparison(bibtex_str)
            corpus_entries[paper_id] = normalized_data
    
    print(f"Normalized {len(corpus_entries)} papers in corpus")
    
    # Step 2: Initialize citation matrix
    paper_ids = list(corpus_entries.keys())
    citation_matrix = pd.DataFrame(0, index=paper_ids, columns=paper_ids)
    
    # Step 3: Extract all references first 
    print("First extracting all references from papers...")
    all_paper_references = {}
    
    # Create output directory for citations
    os.makedirs("extracted_citations", exist_ok=True)
    
    for paper_id in tqdm(corpus_entries.keys()):
        
        time.sleep(1)  # To avoid overwhelming the server
        
        if paper_id in pdf_path_dict:
            pdf_path = f"{path_constant_part}{pdf_path_dict[paper_id]}"
            
            # Extract references
            print(f"\nExtracting references from paper {paper_id}...")
            references = extract_references_to_bibtex(pdf_path)
            print(f"Found {len(references)} references in paper {paper_id}")
            
            # Save references to file
            with open(f"extracted_citations/paper_{paper_id}_citations.bib", "w", encoding="utf-8") as f:
                for ref in references:
                    f.write(ref['bibtex'] + "\n\n")
            
            all_paper_references[paper_id] = references
    
    # Step 4: Match references to corpus and collect uncertain matches
    print("\nMatching references to corpus papers...")
    citation_details = {}
    uncertain_matches = []  # Store uncertain matches for Excel confirmation
    high_confidence_matches = []  # Store high confidence matches
    
    for citing_id, references in all_paper_references.items():
        citation_details[citing_id] = []
        
        for ref_idx, ref in enumerate(references):
            matches = match_citation_to_corpus(ref, corpus_entries, citing_id)
            
            if matches:
                best_match = matches[0]
                cited_id = best_match['paper_id']
                match_score = best_match['score']
                
                # High-confidence match (add directly)
                if match_score >= 0.7:
                    citation_matrix.loc[citing_id, cited_id] = 1
                    high_confidence_matches.append({
                        'citing_id': citing_id,
                        'cited_id': cited_id,
                        'score': match_score,
                        'citation_title': ref.get('title', ''),
                        'citation_authors': ', '.join(ref.get('authors', [])),
                        'citation_year': ref.get('year', ''),
                        'confidence': 'high',
                        'matching_authors': ', '.join(best_match.get('details', {}).get('matching_authors', [])),
                        'title_similarity': best_match.get('details', {}).get('title_similarity', 0),
                        'year_match': 'Yes' if best_match.get('details', {}).get('year_match', False) else 'No',
                        'confirmed': 'Yes'  # Auto-confirmed due to high confidence
                    })
                    print(f"✓ Paper {citing_id} cites paper {cited_id} (score: {match_score:.2f})")
                
                # Uncertain match (store for Excel confirmation)
                elif match_score >= 0.5:
                    citing_filename = pdf_path_dict.get(citing_id, f"Unknown-{citing_id}")
                    cited_filename = pdf_path_dict.get(cited_id, f"Unknown-{cited_id}")
                    
                    uncertain_matches.append({
                        'citing_id': citing_id,
                        'citing_filename': citing_filename,
                        'cited_id': cited_id,
                        'cited_filename': cited_filename,
                        'score': match_score,
                        'citation_title': ref.get('title', ''),
                        'citation_authors': ', '.join(ref.get('authors', [])),
                        'citation_year': ref.get('year', ''),
                        'confidence': 'medium',
                        'matching_authors': ', '.join(best_match.get('details', {}).get('matching_authors', [])),
                        'title_similarity': best_match.get('details', {}).get('title_similarity', 0),
                        'year_match': 'Yes' if best_match.get('details', {}).get('year_match', False) else 'No',
                        'confirmed': ''  # To be filled in Excel
                    })
                    print(f"? Uncertain match: Paper {citing_id} possibly cites paper {cited_id} (score: {match_score:.2f})")
    
    # Step 5: Export uncertain matches to Excel
    if uncertain_matches:
        print(f"\nExporting {len(uncertain_matches)} uncertain matches to Excel...")
        uncertain_df = pd.DataFrame(uncertain_matches)
        
        # Add instructions in first row
        instructions = pd.DataFrame([{
            'citing_id': 'INSTRUCTIONS',
            'citing_filename': 'Fill in the "confirmed" column with: Yes, No, or leave blank to skip',
            'cited_id': '',
            'cited_filename': '',
            'score': '',
            'citation_title': '',
            'citation_authors': '', 
            'citation_year': '',
            'confidence': '',
            'matching_authors': '',
            'title_similarity': '',
            'year_match': '',
            'confirmed': ''
        }])
        
        # Combine instructions with data
        export_df = pd.concat([instructions, uncertain_df], ignore_index=True)
        
        # Export to Excel
        excel_path = 'citation_confirmation.xlsx'
        export_df.to_excel(excel_path, index=False)
        print(f"Exported uncertain matches to {excel_path}")
        print("Please fill in the 'confirmed' column with 'Yes' or 'No' and save the file.")
        print("Then run the import_citation_confirmations() function to update the citation matrix.")
    
    # Also export high confidence matches for reference
    if high_confidence_matches:
        high_conf_df = pd.DataFrame(high_confidence_matches)
        high_conf_df.to_excel('high_confidence_citations.xlsx', index=False)
    
    return citation_matrix, citation_details, uncertain_matches

In [30]:
def import_citation_confirmations(citation_matrix):
    """Import citation confirmations from Excel and update the citation matrix"""
    confirmation_file = 'citation_confirmation.xlsx'
    
    if not os.path.exists(confirmation_file):
        print(f"Error: Confirmation file {confirmation_file} not found.")
        return citation_matrix
    
    # Load confirmations, skipping the instruction row
    confirmations = pd.read_excel(confirmation_file, header=0, skiprows=[1])
        
    # Count statistics
    confirmed_count = 0
    rejected_count = 0
    skipped_count = 0
    
    # Process each confirmation
    for _, row in confirmations.iterrows():
        citing_id = row['citing_id']
        cited_id = row['cited_id']
        confirmation = str(row['confirmed']).strip().lower()
        
        if confirmation == 'yes':
            citation_matrix.loc[citing_id, cited_id] = 1
            confirmed_count += 1
        elif confirmation == 'no':
            # Ensure it's set to 0 (though it should already be)
            citation_matrix.loc[citing_id, cited_id] = 0
            rejected_count += 1
        else:
            skipped_count += 1
    
    # Print summary
    print(f"\nCitation Confirmation Summary:")
    print(f"- Confirmed: {confirmed_count}")
    print(f"- Rejected: {rejected_count}")
    print(f"- Skipped: {skipped_count}")
    
    # Save updated matrix
    citation_matrix.to_csv('interconnections_datasets/citation_matrix.csv')
    print("Updated citation matrix saved to 'interconnections_datasets/citation_matrix.csv'")
    
    return citation_matrix

In [10]:
# Execute the modified pipeline
citation_matrix, citation_details, uncertain_matches = build_citation_network_with_excel(
    df_id_bibtex, 
    pdf_path_dict, 
    path_constant_part
)

# Save initial citation matrix (with only high-confidence matches)
citation_matrix.to_csv('interconnections_datasets/citation_matrix_initial.csv')

# Display instructions for the user
print("\n" + "="*80)
print("NEXT STEPS:")
print("1. Open the file 'citation_confirmation.xlsx'")
print("2. For each row, review the potential citation")
print("3. In the 'confirmed' column, enter:")
print("   - 'Yes' if it's a valid citation")
print("   - 'No' if it's not a valid citation")
print("   - Leave blank to skip")
print("4. Save the file and run the code below to update the citation matrix:")
print("   citation_matrix = import_citation_confirmations(citation_matrix)")
print("="*80)


Preparing citation network analysis...
Normalizing corpus BibTeX entries...
Normalized 118 papers in corpus
First extracting all references from papers...


  0%|          | 0/118 [00:00<?, ?it/s]


Extracting references from paper 1...


  1%|          | 1/118 [00:45<1:29:21, 45.83s/it]

Found 11 references in paper 1

Extracting references from paper 2...


...

 99%|█████████▉| 117/118 [08:26<00:04,  4.20s/it]

Found 95 references in paper 117

Extracting references from paper 118...


100%|██████████| 118/118 [08:29<00:00,  4.32s/it]

Found 34 references in paper 118

Matching references to corpus papers...


✓ Paper 4 cites paper 5 (score: 1.00)
✓ Paper 5 cites paper 3 (score: 0.88)
✓ Paper 5 cites paper 2 (score: 0.94)
✓ Paper 5 cites paper 4 (score: 1.00)
? Uncertain match: Paper 9 possibly cites paper 7 (score: 0.58)
✓ Paper 11 cites paper 5 (score: 1.00)
✓ Paper 11 cites paper 2 (score: 1.00)
✓ Paper 11 cites paper 6 (score: 1.00)
✓ Paper 11 cites paper 4 (score: 1.00)
✓ Paper 11 cites paper 10 (score: 1.00)
✓ Paper 13 cites paper 6 (score: 1.00)
✓ Paper 14 cites paper 8 (score: 1.00)
✓ Paper 16 cites paper 2 (score: 1.00)
✓ Paper 16 cites paper 5 (score: 1.00)
✓ Paper 16 cites paper 6 (score: 1.00)
✓ Paper 16 cites paper 11 (score: 1.00)
? Uncertain match: Paper 16 possibly cites paper 11 (score: 0.60)
✓ Paper 16 cites paper 12 (score: 1.00)
✓ Paper 16 cites paper 13 (score: 0.86)
✓ Paper 17 cites paper 8 (score: 1.00)
✓ Paper 17 cites paper 5 (score: 1.00)
✓ Paper 17 cites paper 3 (score: 0.86)
? Uncertain match: Paper 17 possibly cites paper 53 (score: 0.57)
? Uncertain match: Paper

In [19]:
# Note: There is an error for the Paper 15 since the PDF seems not well-readable. It cites none of the other studies anyways.

In [31]:
# To run after manual confirmation:
citation_matrix = import_citation_confirmations(citation_matrix)

# Print summary statistics
num_citations = citation_matrix.sum().sum()
num_papers_with_citations = (citation_matrix.sum(axis=1) > 0).sum()
num_papers_cited = (citation_matrix.sum(axis=0) > 0).sum()

print(f"\nUpdated Citation Network Summary:")
print(f"Total citations between corpus papers: {num_citations}")
print(f"Papers that cite others in the corpus: {num_papers_with_citations}")
print(f"Papers that are cited by others: {num_papers_cited}")


Citation Confirmation Summary:
- Confirmed: 65
- Rejected: 183
- Skipped: 0
Updated citation matrix saved to 'interconnections_datasets/citation_matrix.csv'

Updated Citation Network Summary:
Total citations between corpus papers: 465
Papers that cite others in the corpus: 96
Papers that are cited by others: 90


In [32]:
# Identify cases where a paper cites another with a higher ID (could be due to same year; want to avoid abvious logical errors)
forward_citations = []

# Iterate through the citation matrix
for citing_id in citation_matrix.index:
    for cited_id in citation_matrix.columns:
        # Check if this is a forward citation (earlier paper citing later paper)
        if citing_id < cited_id and citation_matrix.loc[citing_id, cited_id] == 1:
            forward_citations.append((citing_id, cited_id))

# Print the results
print(f"Found {len(forward_citations)} forward citations (earlier papers citing later papers):")
if forward_citations:
    for citing_id, cited_id in forward_citations:
        print(f"Paper {citing_id} cites paper {cited_id}")
else:
    print("No forward citations found.")

Found 13 forward citations (earlier papers citing later papers):
Paper 4 cites paper 5
Paper 46 cites paper 68
Paper 54 cites paper 68
Paper 59 cites paper 60
Paper 61 cites paper 63
Paper 61 cites paper 64
Paper 61 cites paper 68
Paper 76 cites paper 77
Paper 93 cites paper 104
Paper 106 cites paper 109
Paper 106 cites paper 111
Paper 108 cites paper 110
Paper 112 cites paper 117


In [33]:
# List of specific forward citations to remove
citations_to_correct = [
    # List the 7 specific pairs you want to remove as (citing_id, cited_id)
    # For example:
    (4, 5),
    (46, 68),
    (54, 68),
    (59, 60),
    (61, 64),
    (61, 68),
    (93, 104)
    # Add all 7 pairs here
]

# Make a copy of the original citation matrix to avoid modifying the original
corrected_matrix = citation_matrix.copy()

# Modify the specified citations
for citing_id, cited_id in citations_to_correct:
    print(f"Removing citation: Paper {citing_id} → Paper {cited_id}")
    corrected_matrix.loc[citing_id, cited_id] = 0

# Save the corrected citation matrix
corrected_matrix.to_csv('interconnections_datasets/citation_matrix_corrected.csv')

# Verify the corrections
forward_citations_after = []
for citing_id in corrected_matrix.index:
    for cited_id in corrected_matrix.columns:
        if citing_id < cited_id and corrected_matrix.loc[citing_id, cited_id] == 1:
            forward_citations_after.append((citing_id, cited_id))

print(f"\nAfter correction: {len(forward_citations_after)} forward citations remain.")
if forward_citations_after:
    print("Remaining forward citations:")
    for citing_id, cited_id in forward_citations_after:
        print(f"Paper {citing_id} still cites paper {cited_id}")
else:
    print("All specified forward citations have been removed.")

Removing citation: Paper 4 → Paper 5
Removing citation: Paper 46 → Paper 68
Removing citation: Paper 54 → Paper 68
Removing citation: Paper 59 → Paper 60
Removing citation: Paper 61 → Paper 64
Removing citation: Paper 61 → Paper 68
Removing citation: Paper 93 → Paper 104

After correction: 6 forward citations remain.
Remaining forward citations:
Paper 61 still cites paper 63
Paper 76 still cites paper 77
Paper 106 still cites paper 109
Paper 106 still cites paper 111
Paper 108 still cites paper 110
Paper 112 still cites paper 117
